# SpaceX Falcon 9 First Stage Landing Prediction

## Launch Sites Locations Analysis with Folium

The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.

In the previous exploratory data analysis, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn`, and discovered some preliminary correlations between the launch site and success rates. Here you will be performing more interactive visual analytics using `Folium`.

* Mark all launch sites on a map.
* Mark the success/failed launches for each site on the map.
* Calculate the distances between a launch site to its proximities.

After completed the above tasks, you should be able to find some geographical patterns about launch sites.

In [1]:
import folium
import pandas as pd

In [2]:
from folium.plugins import MarkerCluster, MousePosition
from folium.features import DivIcon

In [3]:
df = pd.read_csv('spacex_launch_geo.csv')
df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [4]:
df = df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = df.groupby('Launch Site', as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df.head()

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.

We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.

In [5]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate.

Now, let's add a circle for each launch site in data frame `launch_sites`

In [6]:
for idx in range(len(launch_sites_df)):
  # Obtaining individual launch site name and its coordinates [Lat, Long]
  launch_site_name = launch_sites_df.loc[idx, 'Launch Site']
  coordinate = [launch_sites_df.loc[idx, 'Lat'], launch_sites_df.loc[idx, 'Long']]

  # Adding a Circle
  circle = folium.Circle(location=coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup(launch_site_name))

  # Adding a Marker
  marker = folium.map.Marker(coordinate, 
                            icon=DivIcon(
                                icon_size=(20,20),
                                icon_anchor=(0,0), 
                                html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % launch_site_name,
                                )
                              )
  site_map.add_child(circle)
  site_map.add_child(marker)

site_map

As you can see from this interactive plot,
1. All the launch sites are closer to the Equator line.
2. All the launch sites are in very close proximity to the coast.

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates. Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not

In [7]:
df.tail()

,Launch Site,Lat,Long,class
51,CCAFS SLC-40,28.563197,-80.57682,0
52,CCAFS SLC-40,28.563197,-80.57682,0
53,CCAFS SLC-40,28.563197,-80.57682,0
54,CCAFS SLC-40,28.563197,-80.57682,1
55,CCAFS SLC-40,28.563197,-80.57682,0


Next, let's create markers for all launch records.
* If a launch was successful `(class=1)`, then we use a green marker.
* If a launch was failed, we use a red marker `(class=0)`.

Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.

Let's first create a `MarkerCluster` object

In [8]:
marker_cluster = MarkerCluster()

In [9]:
df['marker_color'] = df['class'].map(lambda x:'green' if x==1 else 'red')
df.tail()

,Launch Site,Lat,Long,class,marker_color
51,CCAFS SLC-40,28.563197,-80.57682,0,red
52,CCAFS SLC-40,28.563197,-80.57682,0,red
53,CCAFS SLC-40,28.563197,-80.57682,0,red
54,CCAFS SLC-40,28.563197,-80.57682,1,green
55,CCAFS SLC-40,28.563197,-80.57682,0,red


In [10]:
site_map.add_child(marker_cluster)

for idx, record in df.iterrows():
  marker = folium.Marker([record['Lat'], record['Long']],
                         icon=folium.Icon(color='white',
                                          icon_color=record['marker_color']
                                          )
                         )
  marker_cluster.add_child(marker)

site_map

Next, we need to explore and analyze the proximities of launch sites.

Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)

In [11]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.

In [12]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [13]:
# Finding coordinate closest to coastline, eg: Lat: 28.56367  Lon: -80.57163
launch_site_lat = 28.56946
launch_site_lon = -80.65307
coastline_lat = 28.58152
coastline_lon = -80.57337

distance = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [14]:
distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
        )
    )

lines = folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]],
    weight=1
)

site_map.add_child(lines)
site_map

In [15]:
railroad_lat = 28.57175
railroad_lon = -80.6539
distance = calculate_distance(launch_site_lat, launch_site_lon, railroad_lat, railroad_lon)

distance_marker = folium.Marker(
    [railroad_lat, railroad_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
        )
    )

lines = folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [railroad_lat, railroad_lon]],
    weight=1
)

site_map.add_child(lines)

In [16]:
highway_lat = 28.57142
highway_lon = -80.65542
distance = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)

distance_marker = folium.Marker(
    [highway_lat, highway_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
        )
    )

lines = folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [highway_lat, highway_lon]],
    weight=1
)

site_map.add_child(lines)

In [17]:
city_melbourne_lat = 28.09186
city_melbourne_lon = -80.64508
distance = calculate_distance(launch_site_lat, launch_site_lon, city_melbourne_lat, city_melbourne_lon)

distance_marker = folium.Marker(
    [city_melbourne_lat, city_melbourne_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
        )
    )

lines = folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [city_melbourne_lat, city_melbourne_lon]],
    weight=1
)

site_map.add_child(lines)
site_map

We can clearly see that,

1. Launch Sites are close to the coastlines.
2. Launch Sites are close to the highways.
3. Launch Sites are close to the railways.
4. Launch Sites keep their distance from the cities to avoid accidental damage and to increase saftey.

***